In [ ]:
import pandas as pd
import numpy as np
import math
import random
import gzip
import operator
from urllib.parse import unquote

import scipy.stats as stats
import statsmodels.formula.api as smf

import networkx as nx

from IPython.display import Image
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.datasets import make_blobs, make_moons, make_gaussian_quantiles
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, classification_report, confusion_matrix, roc_auc_score, average_precision_score, balanced_accuracy_score,  mean_squared_error, mean_absolute_error, auc, roc_curve
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,  cross_val_predict, cross_val_score
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegressionCV, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.sql.window import Window

pd.options.mode.chained_assignment = None 

### Tutorials
#### Tutorial 01 - Handling data (Introduction to Pandas)
plot, index, sort, loc, merge, concat, reshape, dummies, categories, groupby, apply

#### Tutorial 02 - Data Viz and Data from the Web
all the visualizations tools with dataframe (plot, subplots, heatmap, ...)

#### Tutorial 03 - Describing data
statistics, test significance, multiple example to plot regressions also 

#### Tutorial 04 - Regression analysis
regression with the operands (+, ~, ...), OLS, logistic regression, odds, log model

#### Tutorial 05 - Observational studies
Similarity, propensity_score 

#### Tutorial 06 - Supervised Learning
OLS, K-NN, LogisticRegression, Random Forest

#### Tutorial 07 - Applied Machine Learning
split train/test, create categories, confusion matrix, compute all possible scores and plot for them

#### Tutorial 08 - Unsupervised Learning
K-NN (vary k), silhouette score,elbow method, PCA, t-SNE, DBSCAN 

#### Tutorial 09 - Handing text
ça skip à l'exa ce subject


#### Tutorial 10 - Handling Networks
Graphs 

#### Tutorial 11 - Scaling Up 
pyspark 
 

**Import Files**

In [ ]:
pd.read_csv?

In [ ]:
df= pd.read_csv("data/article_df_task-A.tsv.gz", # path
                 compression="infer",  # handles compression
                 sep="\t",             # handles .tsv
                 error_bad_lines=False # handles bad lines
)

In [ ]:
tweets_data_path = './Data/twitter_data.txt'
tweets_data = []
with open(tweets_data_path, "r") as tweets_file:
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue

In [ ]:
with open('./files/info.txt', 'r') as file:
    print(file.read())

**Print**

Use %d for decimals/integers, %f for floats (alternatively %.xf to specify a precision of x), and %s for strings

In [ ]:
print("There are %d apples on the table." % (num_off_apples))

**GroupBy**

In [ ]:
pd.DataFrame.groupby?

**Statistics**

p_value < 0.05 -> we can reject the null hypothesis

In [ ]:
# Correlation
stats.spearmanr?

# Plot regression
sn.lmplot?

# Test if normal dist but can test other dist
diagnostic.kstest_normal? 

# T-test for check if two mean same
stats.ttest_ind?


##### CF with bootstrapping

In [ ]:
def do_bootstrap(data, n=1000):
    sample_statistic = [] 
    for _ in range(n):
        sampled_data = np.random.choice(data, size=len(data))  
        sample_statistic.append(np.mean(sampled_data))
    return (np.percentile(sample_statistic, 2.5), np.percentile(sample_statistic, 97.5))

**Regression**

- Equations are specified using patsy formula syntax. Important operators are:
    1. `~` : Separates the left-hand side and right-hand side of a formula.
    2. `+` : Creates a union of terms that are included in the model.
    3. `:` : Interaction term.
    3. `*` : `a * b` is short-hand for `a + b + a:b`, and is useful for the common case of wanting to include all interactions between a set of variables.
    
    
- Intercepts are added by default.


- Categorical variables can be included directly by adding a term C(a).

##### Linear Regression

In [ ]:
# Declares the model
mod = smf.ols(formula='time ~ C(diabetes) + C(high_blood_pressure)', data=df)
# Fits the model (find the optimal coefficients, adding a random seed ensures consistency)
np.random.seed(2)
res = mod.fit()
# Print thes summary output provided by the library.
print(res.summary())

##### Logistic regression (binary)

In [ ]:
# Need to standardize the countinuous variables
df['age'] = (df['age'] - df['age'].mean())/df['age'].std()
# Logit is logistic regression.
mod = smf.logit(formula='DEATH_EVENT ~  age + creatinine_phosphokinase + ejection_fraction + \
                        platelets + serum_creatinine + serum_sodium + \
                        C(diabetes) + C(high_blood_pressure) +\
                        C(sex) + C(anaemia) + C(smoking) + C(high_blood_pressure)', data=df)
res = mod.fit()
print(res.summary())

In [ ]:
# feature names
variables = res.params.index
# coefficients
coefficients = res.params.values
# p-values
p_values = res.pvalues
# standard errors
standard_errors = res.bse.values
#confidence intervals
res.conf_int()

**Unsupervised Learning**

##### Linear Regression

In [ ]:
lin_reg = LinearRegression()  # create the model
lin_reg.fit(X, y)  # train it

# see the formula of the regression with the coefficient found
for f in range(len(feature_cols)):
    print("{0} * {1} + ".format(lin_reg.coef_[f], feature_cols[f]))
print(lin_reg.intercept_)

# cross_val_predict returns an array of the same size as `y` where each entry is a prediction obtained by cross validation:
predicted = cross_val_predict(lr, X, y, cv=5)

#MSE
mean_squared_error(y, predicted)

##### Ridge Regression

puts a penalty on large weights Beta and forces them to be smaller in magnitude. This reduces the complexity of the model.

In [ ]:
ridge = Ridge(alpha=6)
predicted_r = cross_val_predict(ridge, X, y, cv=5)

##### LogisticRegresion

Logistic regression uses a threshold on the probability to decide at which class to assign a prediction. In some cases, we are interested to understand how the model behaves at different levels of this threshold (ROC curve)

Feature Vectors (dummies)

In [ ]:
# The features vector
X = pd.get_dummies(titanic[titanic_features])

# Can fill empty rows/cells
X = X.fillna(X.mean())

In [ ]:
logistic = LogisticRegression(solver='lbfgs')

# Possible way to see utput for one test
"YES" if logistic.predict([test])[0] > 0 else "NO"
# Probability decision for a test 
logistic.predict_proba([test])

Can change a "continous" vector to a "discrete" vector

In [ ]:
threshold = y_train.median()

y_train_binary = (y_train > threshold).astype(int)
y_test_binary = (y_test > threshold).astype(int)

###### Logisitic Regression Topic classification (multi-class)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

parameters = {
    'clf__alpha': [1e-4],    
}

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(penalty='l2', loss='log', max_iter=5, tol=None, random_state=42))
])


gs_clf = GridSearchCV(text_clf, parameters, cv=5)
gs_clf = gs_clf.fit(X_train, y_train)

print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
    
predicted = gs_clf.predict(X_test)
print("Accuracy on Test Data: ", np.mean(predicted == y_test))

##### Ridge Regression

Technique for analyzing multiple regression data that suffer from multicollinearity.

Find and use the optimal regularization parameter $\alpha$ from the set {0.001, 0.01, 0.1} via 3-fold cross validation.

In [ ]:
ridge = Ridge()
ridge_hyper = {'alpha':(0.001, 0.01, 0.1)}
ridge_cv = GridSearchCV(ridge, ridge_hyper, cv=3)
ridge_cv.fit(X_train, y_train)

ridge_cv.cv_results_['mean_test_score']

mean_absolute_error(y_test, ridge_cv.predict(X_test))

Precision and Recall

In [ ]:
precision = cross_val_score(logistic, X, y, cv=10, scoring="precision")
recall = cross_val_score(logistic, X, y, cv=10, scoring="recall")

# Precision: avoid false positives
print("Precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std() * 2))
# Recall: avoid false negatives
print("Recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std() * 2))

##### K-NN

In [ ]:
# K = 1
clf_moons_1 = KNeighborsClassifier(1)
clf_moons_1.fit(X_moons, y_moons)
clf_circles_1 = KNeighborsClassifier(1)
clf_circles_1.fit(X_circles, y_circles)

##### DBSCAN

In [ ]:
eps_list = np.linspace(0.05, 0.15, 14)
labels = DBSCAN(eps=eps).fit_predict(X_moons)

##### Random Forest Model

- Use random forest classifier with max tree depth of 3 (and random_state=0)
- Train the classifier by variating the number of trees from 1 to 20 (N)
- For each step estimate precision/recall with cross validation (10-folds)


In [ ]:
number_trees = [n for n in range(1, 21)]
precision_scores = []
recalls_scores = []


for nt in number_trees:
    clf = RandomForestClassifier(max_depth=3, random_state=0, n_estimators=nt)
    clf.fit(X, y)
    precision = cross_val_score(clf, X, y, cv=10, scoring="precision")
    precision_scores.append(precision.mean())
    recall = cross_val_score(clf, X, y, cv=10, scoring="recall")
    recalls_scores.append(recall.mean())

**Observational studies**

**Propensity score** of a data point represents its probability of receiving the treatment, based on its pre-treatment features (in this case, age, education, pre-treatment income, etc.)

**Observational studies** are ones where researchers observe the effect of a risk factor, diagnostic test, treatment or other intervention without trying to change who is or isn't exposed to it. Cohort studies and case control studies are two types of observational studies

##### Balancing the dataset via matching (Graph)

$$ similarity(x,y) = 1 - | propensity\_score(x) - propensity\_score(y) |$$

In [ ]:
def get_similarity(propensity_score1, propensity_score2):
    '''Calculate similarity for instances with given propensity scores'''
    return 1-np.abs(propensity_score1-propensity_score2)

In [ ]:
# Separate the treatment and control groups
treatment_df = lalonde_data[lalonde_data['treat'] == 1]
control_df = lalonde_data[lalonde_data['treat'] == 0]

# Create an empty undirected graph
G = nx.Graph()

# Loop through all the pairs of instances
for control_id, control_row in control_df.iterrows():
    for treatment_id, treatment_row in treatment_df.iterrows():

        # Calculate the similarity 
        similarity = get_similarity(control_row['Propensity_score'],
                                    treatment_row['Propensity_score'])

        # Add an edge between the two instances weighted by the similarity between them
        G.add_weighted_edges_from([(control_id, treatment_id, similarity)])

# Generate and return the maximum weight matching on the generated graph
matching = nx.max_weight_matching(G)

matched = [i[0] for i in list(matching)] + [i[1] for i in list(matching)]

# new datafram with only the matched pairs
balanced_df_1 = lalonde_data.iloc[matched]

#can add conditions to have a better match if for example a feature is not balanced (in this tutorial, it's race)

**Processing Dataset before ML** 

##### Train/Test

In [ ]:
def split_set(data_to_split, ratio=0.8):
    mask = np.random.rand(len(data_to_split)) < ratio
    return [data_to_split[mask].reset_index(drop=True), data_to_split[~mask].reset_index(drop=True)]

In [ ]:
# Without X,y ... "only" X
train, test = train_test_split(youtube_ml, test_size=0.3, random_state=42)

# With X,y 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

##### Categories

In [ ]:
categorical_columns = ['sex_upon_outcome', 'animal_type', 'intake_condition',
                       'intake_type', 'sex_upon_intake']
train_categorical = pd.get_dummies(train, columns=categorical_columns)
train_categorical.columns

##### Label/Features (y, X)

In [ ]:
train_label=train_categorical.adopted
train_features = train_categorical.drop('adopted', axis=1)

test_label=test_categorical.adopted
test_features = test_categorical.drop('adopted', axis=1)


##### Standardize

In [ ]:
means = train_features.mean()
stddevs = train_features.std()

train_features_std = pd.DataFrame()
for c in train_features.columns:
    train_features_std[c] = (train_features[c]-means[c])/stddevs[c]

# Use the mean and stddev of the training set
test_features_std = pd.DataFrame()
for c in test_features.columns:
    test_features_std[c] = (test_features[c]-means[c])/stddevs[c]

train_features_std.head()

**Score ML**

##### Confusion Matrix

In [ ]:
def compute_confusion_matrix(true_label, prediction_proba, decision_threshold=0.5): 
    
    predict_label = (prediction_proba[:,1]>decision_threshold).astype(int)   
                                                                                                                       
    TP = np.sum(np.logical_and(predict_label==1, true_label==1))
    TN = np.sum(np.logical_and(predict_label==0, true_label==0))
    FP = np.sum(np.logical_and(predict_label==1, true_label==0))
    FN = np.sum(np.logical_and(predict_label==0, true_label==1))
    
    confusion_matrix = np.asarray([[TP, FP],
                                    [FN, TN]])
    return confusion_matrix


def compute_all_score(confusion_matrix, t=0.5):
    [[TP, FP],[FN, TN]] = confusion_matrix.astype(float)
    
    accuracy =  (TP+TN)/np.sum(confusion_matrix)
    
    precision_positive = TP/(TP+FP) if (TP+FP) !=0 else np.nan
    precision_negative = TN/(TN+FN) if (TN+FN) !=0 else np.nan
    
    recall_positive = TP/(TP+FN) if (TP+FN) !=0 else np.nan
    recall_negative = TN/(TN+FP) if (TN+FP) !=0 else np.nan

    F1_score_positive = 2 *(precision_positive*recall_positive)/(precision_positive+recall_positive) if (precision_positive+recall_positive) !=0 else np.nan
    F1_score_negative = 2 *(precision_negative*recall_negative)/(precision_negative+recall_negative) if (precision_negative+recall_negative) !=0 else np.nan

    return [t, accuracy, precision_positive, recall_positive, F1_score_positive, precision_negative, recall_negative, F1_score_negative]

##### Silhouette score 
- k vs silhouette score plot
- need to take the highest score

In [ ]:
silhouettes = []

# Try multiple k
for k in range(2, 11):
    # Cluster the data and assigne the labels
    labels = KMeans(n_clusters=k, random_state=10).fit_predict(X)
    # Get the Silhouette score
    score = silhouette_score(X, labels)
    silhouettes.append({"k": k, "score": score})
    
# Convert to dataframe
silhouettes = pd.DataFrame(silhouettes)

##### Elbow method
- sum of squared errors vs k
- quand ça break sur le plot wesh 

In [ ]:
def plot_sse(features_X, start=2, end=11):
    sse = []
    for k in range(start, end):
        # Assign the labels to the clusters
        kmeans = KMeans(n_clusters=k, random_state=10).fit(features_X)
        sse.append({"k": k, "sse": kmeans.inertia_})

**High Dimensional Data**

Dimensionality Reduction reduce the number of dimensions by preserving as much information as possible

##### t-SNE
- It is a non-linear Dimensionality reduction technique
-  It embeds the points from a higher dimension to a lower dimension trying to preserve the neighborhood of that point.

In [ ]:
X_reduced_tsne = TSNE(n_components=2, random_state=0).fit_transform(X10d)

##### PCA 
- It is a linear Dimensionality reduction technique
- The main idea behind this technique is to reduce the dimensionality of data that is highly correlated by transforming the original set of vectors to a new set which is known as Principal component.

In [ ]:
X_reduced_pca = PCA(n_components=2).fit(X10d).transform(X10d)

**Graph**

In [ ]:
G = nx.Graph() # for a directed graph use nx.DiGraph()
nx.info(G)

# add node attributes by passing dictionary of type name -> attribute
nx.set_node_attributes(quakerG, nodes['Role'].to_dict(), 'Role' )

#Create graph from numpy array
g_text = networkx.from_numpy_array(g_text_adj)

#Find diameter of a graph (largest shortest-path length, where the maximization is done over all node pairs)
networkx.diameter(g_text)

In [ ]:
# You can easily get the attributes of a node
quakerG.nodes['William Penn']

##### Degree 

In [ ]:
degrees = dict(quakerG.degree(quakerG.nodes()))
sorted_degree = sorted(degrees.items(), key=itemgetter(1), reverse=True)

##### Betweeness Centrality

the more shortest paths pass through a node, the more important it is

In [ ]:
# Compute betweenness centrality
betweenness = nx.betweenness_centrality(quakerG)
# Assign the computed centrality values as a node-attribute in your network
nx.set_node_attributes(quakerG, betweenness, 'betweenness')
sorted_betweenness = sorted(betweenness.items(), key=itemgetter(1), reverse=True)

for quaker, bw in sorted_betweenness[:5]:
    print(quaker, 'who is', quakerG.nodes[quaker]['Role'], 'has betweeness: %.3f' %bw)

##### Sparsity

$L = \frac{|E|}{|E_{max}|}$, where $E_{max} = \frac{n * (n-1)}{2}$

In [ ]:
nx.density(quakerG)

##### Connected Components

In [ ]:
#Find the number of connected components
networkx.number_connected_components(g_text)

# WC if the underlying undirected graph obtained by replacing all directed edges of the graph with undirected edges is a connected graph 
nx.is_weakly_connected(G)
nx.weakly_connected_components(G)

# SC if, for every pair of vertices $(u, v)$, it contains a directed path from $u$ to $v$ and a directed path from $v$ to $u$.
nx.is_strongly_connected(G)
nx.strongly_connected_components(G)

largest_cc = max(nx.weakly_connected_components(G), key=len)
H = G.subgraph(list(largest_cc))

##### Diameter and Shortest Paths

In [ ]:
fell_whitehead_path = nx.shortest_path(quakerG, source="Margaret Fell", target="George Whitehead")

##### Transitivity

Global clustering coefficient, or the ratio of all existing triangles (closed triples) over all possible triangles (open and closed triplets).

In [ ]:
nx.transitivity(quakerG)

##### Louvain Method

Community detection is a method to extract communities from large networks

In [ ]:
partition = community_louvain.best_partition(quakerG)
# add it as an attribute to the nodes
for n in quakerG.nodes:
    quakerG.nodes[n]["louvain"] = partition[n]

**Spark**

In [ ]:
# create the session
spark = SparkSession.builder.getOrCreate()

# create the context
sc = spark.sparkContext

In [ ]:
Bombing_Operations = spark.read.json("Bombing_Operations.json")

Bombing_Operations.printSchema()

Bombing_Operations.take(3)

#move to pandas
missions_count_pd = missions_counts.toPandas()
missions_count_pd.head()

# select column
missions_aircrafts = missions_joined.select("AirCraftType")

#sort by a specific column
by_posts = subreddit_info.select("subreddit", "total_posts").sort(col("total_posts").desc())

##### GroupBy (SQL or not)

In [ ]:
missions_counts = Bombing_Operations.groupBy("ContryFlyingMission")\
                                    .agg(count("*").alias("MissionsCount"))\
                                    .sort(desc("MissionsCount"))
missions_counts.show()

In [ ]:
Bombing_Operations.registerTempTable("Bombing_Operations")

query = """
SELECT ContryFlyingMission, count(*) as MissionsCount
FROM Bombing_Operations
GROUP BY ContryFlyingMission
ORDER BY MissionsCount DESC
"""

missions_counts = spark.sql(query)
missions_counts.show()

##### Jaccard Similarity

$Jaccard(A,B) = \frac{|A \cap B|}{|A \cup B|}$

In [ ]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

##### Processing text with Pyspark

More on the tutorial's exercice

In [ ]:
# tokenize the text
regexTokenizer = RegexTokenizer(inputCol="body", outputCol="all_words", pattern="\\W")
reddit_with_words = regexTokenizer.transform(reddit)

# remove stop words
remover = StopWordsRemover(inputCol="all_words", outputCol="words")
reddit_with_tokens = remover.transform(reddit_with_words).drop("all_words")

# get all words in a single dataframe
all_words = reddit_with_tokens.select(explode("words").alias("word"))
# group by, sort and limit to 50k 
top50k = all_words.groupBy("word").agg(count("*").alias("total")).sort(col("total").desc()).limit(50000)

top50k.show()